# Quantitative Value Strategy

In [68]:
import numpy as np 
import pandas as pd 
import xlsxwriter
import requests
from scipy import stats 
import math 

In [69]:
stocks = pd.read_csv("constituents.csv")
from secrets import IEX_CLOUD_API_TOKEN
stocks = stocks["Symbol"]

In [70]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [72]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        

##Value Strategy

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)


In [73]:
rv_columns = [
    "Symbol",
    "Price",
    "Number of Shares to Buy",
    "P/E Ratio",
    "P/E Percentile",
    "P/B Ratio",
    "P/B Percentile",
    "P/S Ratio",
    "P/S Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV_Score"
]

Price = {}
peRatio = {}
pbRatio = {}
psRatio = {}
ev = {}
ebitda = {}
gp = {}

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if data[symbol]['quote']['latestPrice'] == None:
            Price[symbol] = -1000
        else: 
            Price[symbol] = data[symbol]['quote']['latestPrice']
        
        if data[symbol]['quote']['peRatio'] == None:
            peRatio[symbol] = -1000
        else:
            peRatio[symbol] = pe_ratio = data[symbol]['quote']['peRatio']
        
        if data[symbol]['advanced-stats']['priceToBook'] == None: 
            pbRatio[symbol] = -1000
        else:
            pbRatio[symbol] = data[symbol]['advanced-stats']['priceToBook']
            
        if data[symbol]['advanced-stats']['priceToSales'] == None:
            psRatio[symbol] = -1000
        else:
            psRatio[symbol] = ps_ratio = data[symbol]['advanced-stats']['priceToSales']
            
        if data[symbol]['advanced-stats']['enterpriseValue'] == None: 
            ev[symbol] = -10000000000
        else: 
            ev[symbol] = data[symbol]['advanced-stats']['enterpriseValue']
            
        if data[symbol]['advanced-stats']['EBITDA'] == None: 
            ebitda[symbol] = 1
        else: 
            ebitda[symbol] = data[symbol]['advanced-stats']['EBITDA']
        
        
        if data[symbol]['advanced-stats']['grossProfit'] == None:
            gp[symbol] = 1
        else: 
            gp[symbol] = data[symbol]['advanced-stats']['grossProfit']
            
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            Price[symbol],
            "N/A",
            peRatio[symbol],
            "N/A",
            pbRatio[symbol],
            "N/A",
            psRatio[symbol],
            "N/A",
            ev[symbol]/ebitda[symbol],
            "N/A",
            ev[symbol]/ebitda[symbol],
            "N/A",
            "N/A"
        ],
        index = rv_columns),
            ignore_index = True
        )


## Calculating Value Percentiles

In [74]:
metrics = {
    "P/E Ratio":"P/E Percentile",
    "P/B Ratio":"P/B Percentile",
    "P/S Ratio":"P/S Percentile",
    "EV/EBITDA":"EV/EBITDA Percentile",
    "EV/GP":"EV/GP Percentile"
}

for matrix in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[matrix]] = stats.percentileofscore(rv_dataframe[matrix], rv_dataframe.loc[row, matrix]) 
       

In [75]:
from statistics import mean
rv_dataframe

for row in rv_dataframe.index:
    value_percenties = []
    for metric in metrics.keys():
        value_percenties.append(rv_dataframe.loc[row,metric])
    rv_dataframe.loc[row, "RV_Score"] = mean(value_percenties)

## Selecting the 50 Best Value Stocks¶

In [76]:
rv_dataframe.sort_values(by = 'RV_Score', ascending = False, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Symbol,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV_Score
0,LUV,46.150,N/A,28.87,60.3586,2.62,37.3506,1.700,26.1952,663.669609,99.8008,663.669609,99.8008,272.106
1,CDAY,73.310,N/A,-144.07,2.58964,4.86,61.753,10.900,89.243,720.086223,100,720.086223,100,262.372
2,NOW,588.340,N/A,512.53,100,32.47,96.0159,20.280,98.2072,158.182674,99.6016,158.182674,99.6016,176.329
3,DXCM,534.350,N/A,342.24,99.6016,23.12,94.4223,21.300,99.6016,128.333925,99.4024,128.333925,99.4024,128.666
4,VTR,63.870,N/A,509.98,99.8008,2.38,33.4661,6.660,78.8845,23.917655,77.0916,23.917655,77.0916,113.371
5,TSLA,1133.400,N/A,235.16,99.2032,38.23,96.6135,20.660,99.004,119.287041,99.2032,119.287041,99.2032,106.525
6,ENPH,207.370,N/A,197.64,98.8048,63.45,98.4064,20.130,97.9084,110.009417,99.004,110.009417,99.004,100.248
7,KSU,302.370,N/A,293.17,99.4024,7.11,73.3068,9.440,86.8526,20.289081,68.7251,20.289081,68.7251,70.0596
8,FTNT,355.700,N/A,97.23,95.8167,74.92,99.2032,16.910,95.8167,73.535741,98.2072,73.535741,98.2072,67.2263
9,TYL,450.230,N/A,118.11,97.012,8.39,78.2869,12.290,91.8327,88.378796,98.6056,88.378796,98.6056,63.1095


## Calculating the Number of Shares to Buy

In [77]:
portfolio_input()

Enter the value of your portfolio:20


In [78]:
position = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, "Number of Shares to Buy"] = math.floor(position/rv_dataframe.loc[row, "Price"])


In [61]:
rv_dataframe

,Symbol,Price,Number of Shares to Buy,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV_Score
0,ALB,229.700,174140,217.06,99.004,4.6500,60.3586,8.0400,83.7649,3.245410e+01,88.6454,3.245410e+01,88.6454,58.9316
1,ABMD,346.090,115576,116.32,96.8127,10.6700,81.8725,15.2100,94.2231,5.192378e+01,96.6135,5.192378e+01,96.6135,49.2095
2,ANSS,326.380,122556,65.02,91.6335,6.4100,70.3187,15.6000,94.6215,4.913750e+01,96.0159,4.913750e+01,96.0159,37.061
3,ABC,157.910,253308,20.11,39.6414,139.6300,99.8008,0.1473,2.39044,1.000865e+01,29.6813,1.000865e+01,29.6813,35.9809
4,ANET,144.930,275995,54.35,88.8446,11.3800,83.2669,15.4000,94.4223,3.955458e+01,94.4223,3.955458e+01,94.4223,32.0478
5,ADBE,480.070,83321,47.00,84.2629,15.5900,89.6414,14.6400,93.6255,3.789297e+01,93.4263,3.789297e+01,93.4263,30.6032
6,ADI,174.000,229885,59.25,90.239,2.3800,33.4661,10.8100,89.243,3.877635e+01,94.0239,3.877635e+01,94.0239,29.9985
7,AMD,126.190,316982,49.59,86.6534,20.0200,93.0279,9.0700,85.6574,3.486065e+01,91.6335,3.486065e+01,91.6335,29.6803
8,AMT,251.186,159244,45.10,82.8685,22.7900,94.0239,12.4800,92.2311,2.743629e+01,84.2629,2.743629e+01,84.2629,27.0485
9,APTV,129.880,307976,66.79,92.2311,4.2400,56.3745,2.2800,36.1554,3.038267e+01,86.6534,3.038267e+01,86.6534,26.8151


## Formatting and Save into Excel Output


In [79]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [80]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [67]:
writer.save()